In [1]:
import cv2
import numpy as np
from keras.models import load_model
from collections import deque

Using TensorFlow backend.


In [2]:
print(cv2.__version__)
print(cv2.data.haarcascades)

model1=load_model("devnagari.h5")
print("....",model1,"....")

4.1.2
C:\Users\KIIT\Anaconda3\lib\site-packages\cv2\data\
.... <keras.engine.sequential.Sequential object at 0x000001B3D1FE1EF0> ....


In [3]:
letter_count={0:'CHECK',1:'01-KAA',2:'02-KHAA',3:'03-GAA',4:'04-GHAA',5:'05-KNA',6:'06-CHA',7:'07-CHHA',8:'08-JAA',9:'09-JHHA'
              
              ,10:'10-IINYAA',11:'11-TAA(TAMATAR)',12:'12-THAA',13:'13-DAA',14:'14-DHHA',15:'15-ADNA',16:'16-TAA(TABLA)',17:'17-THA',18:'18-DA'
              
              ,19:'19-DHA',20:'20-NAA',21:'21-PAA',22:'22-PHA',23:'23-BAA',24:'24-BHA',25:'25-MAA',26:'26-YWA',27:'27-RAA',28:'28-LAA'
              
              ,29:'29-WAA',30:'30-SHA',31:'31-PETCHIRYAKHA',32:'32-SAA',33:'33-HAA',34:'34-CHYAA',35:'35-TRAA',36:'36-GYAA',37:'37-CHECK'
            }

In [ ]:
def keras_predict (model,image):
    processed=keras_process_image(image)
    print("PROCESSED:",str(processed.shape))
    pred_prob=model.predict(processed)[0]
    pred_class=list(pred_prob).index(max(pred_prob))
    return max(pred_prob),pred_class

def keras_process_image(img):
    image_x=32
    image_y=32
    img=cv2.resize(img,(image_x,image_y))
    img=np.array(img,dtype=np.float32)
    img=np.reshape(img,(-1,image_x,image_y,1))
    return img
               

In [ ]:
cap=cv2.VideoCapture(0)
Lower_Blue=np.array([100,50,50])
Upper_Blue=np.array([130,225,255])
pred_class=0
pts=deque(maxlen=512)
blackboard=np.zeros((480,640,3),dtype=np.uint8)
digit=np.zeros((300,300,3),dtype=np.uint8)
while(cap.isOpened()):
    ret,img=cap.read()
    img=cv2.flip(img,1)
    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(imgHSV,Lower_Blue,Upper_Blue)
    blur=cv2.medianBlur(mask,15)
    blur=cv2.GaussianBlur(blur,(5,5),0)
    thresh=cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
    center=None
    if len(cnts)>=1:
        contour = max(cnts,key=cv2.contourArea)
        if(cv2.contourArea(contour)>250):
            ((x,y),radius)=cv2.minEnclosingCircle(contour)
            cv2.circle(img,(int(x),int(y)),int(radius),(0,255,255),2)
            cv2.circle(img,center,5,(0,0,255),2)
            M=cv2.moments(contour)
            center=(int(M['m10']/M['m00']),int(M['m01']/M['m00']))
            pts.appendleft(center)
            for i in range(1,len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard,pts[i-1],pts[i],(255,255,255),10)
                cv2.line(img,pts[i-1],pts[i],(0,0,255),3)
    elif len(cnts)==0:
        if len(pts)!=[]:
            blackboard_gray=cv2.cvtColor(blackboard,cv2.COLOR_BGR2GRAY)
            blur1=cv2.medianBlur(blackboard_gray,15)
            blur1=cv2.GaussianBlur(blur1,(5,5),0)
            thresh1=cv2.threshold(blur1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            blackboard_cnts=cv2.findContours(thresh1.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts)>=1:
                cnt=max(blackboard_cnts,key=cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt)>250:
                    x,y,w,h=cv2.boundingRect(cnt)
                    digit=blackboard_gray[y:y+h,x:x+w]
                    pred_probab,pred_class=keras_predict(model1,digit)
                    print(pred_class,pred_probab)
        pts=deque(maxlen=512)
        blackboard=np.zeros((480,640,3),dtype=np.uint8)
    cv2.putText(img,"CONV NETWORK: " +str(letter_count[pred_class]),(10,470),
               cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    cv2.imshow("Frame",img)
    cv2.imshow("Contours",thresh)
    k=cv2.waitKey(3)
    if k==27:
        cap.release
        break


49603.0
PROCESSED: (1, 32, 32, 1)
28 1.0
1675.5
PROCESSED: (1, 32, 32, 1)
26 1.0
90343.0
PROCESSED: (1, 32, 32, 1)
6 1.0
62324.5
PROCESSED: (1, 32, 32, 1)
2 1.0
6241.5
PROCESSED: (1, 32, 32, 1)
31 1.0
2244.0
PROCESSED: (1, 32, 32, 1)
33 1.0
5987.0
PROCESSED: (1, 32, 32, 1)
7 1.0
13495.0
PROCESSED: (1, 32, 32, 1)
12 1.0
46501.0
PROCESSED: (1, 32, 32, 1)
35 1.0
48505.0
PROCESSED: (1, 32, 32, 1)
12 1.0
38884.0
PROCESSED: (1, 32, 32, 1)
9 1.0
120368.5
PROCESSED: (1, 32, 32, 1)
33 1.0
1014.0
PROCESSED: (1, 32, 32, 1)
24 1.0
16923.0
PROCESSED: (1, 32, 32, 1)
9 1.0
19269.5
PROCESSED: (1, 32, 32, 1)
33 1.0
5601.5
PROCESSED: (1, 32, 32, 1)
17 1.0
87636.0
PROCESSED: (1, 32, 32, 1)
2 1.0
8606.5
PROCESSED: (1, 32, 32, 1)
31 1.0
4705.5
PROCESSED: (1, 32, 32, 1)
33 1.0
14154.0
PROCESSED: (1, 32, 32, 1)
17 1.0
6068.0
PROCESSED: (1, 32, 32, 1)
24 1.0
20384.5
PROCESSED: (1, 32, 32, 1)
17 1.0
53547.5
PROCESSED: (1, 32, 32, 1)
31 1.0
6578.5
PROCESSED: (1, 32, 32, 1)
27 1.0
11826.5
PROCESSED: (1, 32, 32, 